In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [4]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [55]:
data = np.loadtxt("Si111_30_10.xy").T

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdiraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [56]:
intervall_voigt_1 = np.arange(np.where(data[0] > 25.61)[0][0], np.where(data[0] > 25.68)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [57]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1['voigt_1gamma'].set(0.001,vary=True,expr='')

In [58]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/(err_1))

In [59]:
out_1.plot()
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ')
plt.show()

In [60]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 48
    # data points      = 8
    # variables        = 4
    chi-square         = 5.335
    reduced chi-square = 1.334
[[Variables]]
    voigt_1amplitude:   261.165804 +/- 6.275365 (2.40%) (init= 162.7719)
    voigt_1sigma:       0.01232774 +/- 0.000927 (7.52%) (init= 0.008146744)
    voigt_1center:      25.6390317 +/- 0.000163 (0.00%) (init= 25.63572)
    voigt_1gamma:       0.00838932 +/- 0.001286 (15.33%) (init= 0.001)
    voigt_1fwhm:        0.04439603 +/- 0.003339 (7.52%)  == '3.6013100*voigt_1sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.979 
    C(voigt_1amplitude, voigt_1gamma)  =  0.950 
    C(voigt_1amplitude, voigt_1sigma)  = -0.915 
    C(voigt_1amplitude, voigt_1center)  = -0.284 
    C(voigt_1center, voigt_1gamma)  = -0.264 
    C(voigt_1sigma, voigt_1center)  =  0.226 


### Mulit-Voigt

In [65]:
intervall_voigt_2 = np.arange(np.where(data[0] > 28.35)[0][0], np.where(data[0] > 28.55)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [66]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))
pars_2['voigt_2gamma'].set(0.0001,vary=True,expr='')
pars_2['voigt_3gamma'].set(0.0001,vary=True,expr='')

In [67]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2))

In [70]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ')
plt.show()

In [69]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 92
    # data points      = 24
    # variables        = 8
    chi-square         = 567.445
    reduced chi-square = 35.465
[[Variables]]
    voigt_2sigma:       0.01228410 +/- 0.000881 (7.17%) (init= 0.008146744)
    voigt_2amplitude:   905.505158 +/- 23.34925 (2.58%) (init= 585.2871)
    voigt_2center:      28.4272852 +/- 0.000400 (0.00%) (init= 28.4265)
    voigt_2gamma:       0.00716875 +/- 0.000981 (13.68%) (init= 0.0001)
    voigt_2fwhm:        0.04423887 +/- 0.003172 (7.17%)  == '3.6013100*voigt_2sigma'
    voigt_3sigma:       0.01309456 +/- 0.001369 (10.46%) (init= 0.01086232)
    voigt_3amplitude:   482.461329 +/- 21.57661 (4.47%) (init= 497.8287)
    voigt_3center:      28.5008075 +/- 0.000532 (0.00%) (init= 28.49753)
    voigt_3gamma:       0.00463859 +/- 0.001814 (39.12%) (init= 0.0001)
    voigt_3fwhm:        0.04715757 +/- 0.004932 (10.46%)  == '3.601

# Plots der Spektren für die ersten 4 Aufnahme

In [36]:
data_spektrum = np.loadtxt("Si111_40_30_Ni.xy").T

In [37]:
plt.errorbar(data_spektrum[0],data_spektrum[1],np.sqrt(data_spektrum[1]),fmt='. ',label="Messdaten, mit U = 40 kV \n A = 30mA, mit Ni-Filter")
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.legend(loc='best',fontsize=40)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.show()